<a href="https://colab.research.google.com/github/avetodef/modern-data-analysis-tools-itmo/blob/main/lab2_siad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Лабораторная работа №2 - классификация текстов**

#**Задание 1**

Получаю два жанра, которые буду использовать в работе

In [837]:
import random
lst = ['Christian', 'Country', 'Pop', 'Rock', 'R&B'] 
print('ваши жанры', random.choice(lst), 'и', random.choice(lst)) 

ваши жанры Pop и Pop


Жанры - Christian и Rock

Для начала читаем набор данных:

In [838]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/modern-instruments-for-data-analysing-itmo/dataset.csv")

data.head()

,genre,lyrics,SongInfo
0,Christian,"Who am I, that the Lord of all the earth Woul...",CASTING CROWNS - WHO AM I LYRICS
1,Christian,Glory Revealed By His Wounds He was pierced ...,GLORY REVEALED - BY HIS WOUNDS LYRICS
2,Christian,Lord of heaven and earth Lord of all creation...,CAEDMON'S CALL - GOD OF WONDERS LYRICS
3,Christian,I can only imagine what it will be like When ...,MERCYME - I CAN ONLY IMAGINE LYRICS
4,Christian,I am not skilled to understand What God has w...,AARON SHUST - MY SAVIOR MY GOD LYRICS


In [839]:
columns = data[['genre', 'lyrics']]
columns

,genre,lyrics
0,Christian,"Who am I, that the Lord of all the earth Woul..."
1,Christian,Glory Revealed By His Wounds He was pierced ...
2,Christian,Lord of heaven and earth Lord of all creation...
3,Christian,I can only imagine what it will be like When ...
4,Christian,I am not skilled to understand What God has w...
...,...,...
553,R&B,"Ha I dont care ha, about your past I just wan..."
554,R&B,Hoverin by my suitcase Tryin to find a warm ...
555,R&B,I dont know why I love you like I do After a...
556,R&B,"C. C. Rider Elvis Presley Well now see., C. ..."


выводим уникальные значения столбца жанра, чтобы узнать, какие есть жанры

In [840]:
columns['genre'].unique()

array(['Christian', 'Country', 'Hip-Hop', 'Pop', 'Rock', 'R&B'],
      dtype=object)

прочитаем сколько всего песен каждого жанра

In [841]:
columns['genre'].value_counts()

Pop          100
Rock          95
Christian     94
Hip-Hop       91
R&B           91
Country       87
Name: genre, dtype: int64

рандомайзер определил мне жанры Pop и Country. оставляю только их

In [842]:
columns = columns[(columns.genre == 'Christian') | (columns.genre == 'Rock')]

привожу все слова в текстах оставленных жанров к нижнему регистру

In [843]:
lowered = columns['lyrics'].str.lower()
columns['lowered'] = lowered

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


разбиваю тексты на отдельные слова - провожу токенизацию

In [844]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [845]:
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


удаляю стоп-слова

In [846]:
nltk.download('stopwords')
from nltk.corpus import stopwords
noise = stopwords.words('english')

withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])

without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))

columns['without_stop'] = without_stop

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


привожу все слова к исходной форме (лемматизация)

In [847]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [848]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])

In [849]:
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))

In [850]:
columns['lemmatized'] = lemma

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


посмотрим на полученные даные:

In [851]:
columns

,genre,lyrics,lowered,tokened,without_stop,lemmatized
0,Christian,"Who am I, that the Lord of all the earth Woul...","who am i, that the lord of all the earth woul...","[who, am, i, ,, that, the, lord, of, all, the,...",",, lord, earth, would, care, know, name, would...",",, lord, earth, would, care, know, name, would..."
1,Christian,Glory Revealed By His Wounds He was pierced ...,glory revealed by his wounds he was pierced ...,"[glory, revealed, by, his, wounds, he, was, pi...","glory, revealed, wounds, pierced, transgressio...","glory, revealed, wounds, pierced, transgressio..."
2,Christian,Lord of heaven and earth Lord of all creation...,lord of heaven and earth lord of all creation...,"[lord, of, heaven, and, earth, lord, of, all, ...","lord, heaven, earth, lord, creation, lord, hea...","lord, heaven, earth, lord, creation, lord, hea..."
3,Christian,I can only imagine what it will be like When ...,i can only imagine what it will be like when ...,"[i, can, only, imagine, what, it, will, be, li...","imagine, like, walk, side, imagine, eyes, see,...","imagine, like, walk, side, imagine, eyes, see,..."
4,Christian,I am not skilled to understand What God has w...,i am not skilled to understand what god has w...,"[i, am, not, skilled, to, understand, what, go...","skilled, understand, god, willed, ,, god, plan...","skilled, understand, god, willed, ,, god, plan..."
...,...,...,...,...,...,...
462,Rock,Cant explain all the feelings that youre maki...,cant explain all the feelings that youre maki...,"[cant, explain, all, the, feelings, that, your...","cant, explain, feelings, youre, making, feel, ...","cant, explain, feelings, youre, making, feel, ..."
463,Rock,"One foot on the brake and one on the gas, hey...","one foot on the brake and one on the gas, hey...","[one, foot, on, the, brake, and, one, on, the,...","one, foot, brake, one, gas, ,, hey, well, ,, t...","one, foot, brake, one, gas, ,, hey, well, ,, t..."
464,Rock,Carry on my wayward son Therell be peace when...,carry on my wayward son therell be peace when...,"[carry, on, my, wayward, son, therell, be, pea...","carry, wayward, son, therell, peace, done, lay...","carry, wayward, son, therell, peace, done, lay..."
465,Rock,"Ooh yeah Turn it up Come on Im working hard,...","ooh yeah turn it up come on im working hard,...","[ooh, yeah, turn, it, up, come, on, im, workin...","ooh, yeah, turn, come, im, working, hard, ,, y...","ooh, yeah, turn, come, im, working, hard, ,, y..."


разделяем данные на обучающую и тестовую выборки:

In [852]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.genre, train_size = 0.7)

In [853]:
columns.genre.value_counts()

Rock         95
Christian    94
Name: genre, dtype: int64

импортируем библиотеку для вектрозиации - представления текста в численном виде для использования данных на модели

In [854]:
from sklearn.feature_extraction.text import CountVectorizer

используем все варианты значения n - униграммы, биграммы и триграммы

In [855]:
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

для классификации используем Наивный Байесовскй Классификатор

импортируем классификатор

In [856]:
from sklearn.naive_bayes import MultinomialNB 

In [857]:
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)

MultinomialNB()

векторизируем выборку

In [858]:
vectorized_x_test = vectorizer.transform(x_test)

посмотрим предсказания для выборки

In [859]:
clf.predict(vectorized_x_test)

array(['Rock', 'Rock', 'Christian', 'Christian', 'Rock', 'Christian',
       'Christian', 'Christian', 'Rock', 'Rock', 'Rock', 'Rock',
       'Christian', 'Christian', 'Christian', 'Rock', 'Rock', 'Rock',
       'Christian', 'Rock', 'Rock', 'Rock', 'Christian', 'Rock', 'Rock',
       'Christian', 'Christian', 'Rock', 'Rock', 'Christian', 'Christian',
       'Christian', 'Rock', 'Rock', 'Christian', 'Rock', 'Christian',
       'Rock', 'Rock', 'Christian', 'Christian', 'Christian', 'Christian',
       'Rock', 'Christian', 'Rock', 'Rock', 'Rock', 'Christian',
       'Christian', 'Christian', 'Rock', 'Christian', 'Rock', 'Rock',
       'Rock', 'Christian'], dtype='<U9')

получаем оценку классификации

In [860]:
from sklearn.metrics import * 
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

   Christian       0.93      0.81      0.86        31
        Rock       0.80      0.92      0.86        26

    accuracy                           0.86        57
   macro avg       0.86      0.86      0.86        57
weighted avg       0.87      0.86      0.86        57



показатель accuracy равен 0.93 ура победа

# **Задание 2**

добавляем тексты песен

In [861]:
song1_lyrics = '''Foo Fighters The Colour And The Shape Everlong Hello  Ive waited here for you  Everlong   Tonight   I throw myself into  and out of the red out of her head she sang   Come down and waste away with me  down with me  Slow out you wanted it to be  over my head, out of my head she sang  and I wonder  when I sing along with you  If everything could ever feel this real forever  If anything could ever be this good again  the only thing Ill ever ask of you  youve got to promise not to stop when I say when  she sang  Breathe out  so I can breathe you in  Hold you in   and now  I know youve always been  out of your head out of my head i sang   and I wonder  If everything could ever feel this real forever  If anything could ever be this good again  the only thing Ill ever ask of you  youve got to promise not to stop when I say when '''

song2_lyrics = ''' Who taught the sun where to stand in the morning? And who told the ocean you can only come this far? And who showed the moon where to hide till evening? Whose words alone can catch a falling star? Well I know my Redeemer lives I know my Redeemer lives All of creation testifies this life within me cries  I know my Redeemer lives ye-e-eah The very same God that spins things in orbit runs to the weary, the worn and the weak and the same gentle hands that hold me when Im broken  they conquered death to bring me victory Now I know, my Redeemer lives I know my Redeemer lives let all creation testify Let this life within me cry I-I-I know My Redeemer  He lives to take away my shame and He lives forever Ill proclaim that the payment for my sin  was the precious life He gave and now Hes alive and theres an empty grave! And I know  my Redeemer lives He lives I know my Redeemer lives  let all creation testify Let this life within me cry I-I-I know my Redeemer I know my Redeemer lives I know my Redeemer lives I know, That I know, that I know, that i know, that I know He lives Because He lives I can face tomorrow  He lives I know, I know He lives I spoke with Him this morning! He lives The tomb is empty He lives Im going to tell everybody!'''

song1_genre = "Rock"
song2_genre = "Cristian"

создаем датасет из вышеуказанных элементов

In [862]:
songs_dataset = pd.DataFrame(data = {"genre":[song2_genre,song1_genre],"lyrics":[song2_lyrics,song1_lyrics]})

приводим к нижнему регистру

In [863]:
lowered = songs_dataset['lyrics'].str.lower()
songs_dataset['lowered'] = lowered

разбиваем на отдельные слова:

In [864]:
tokened = songs_dataset.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
songs_dataset['tokened'] = tokened

удаляем стоп-слова:

In [865]:
withoutstop = songs_dataset['tokened'].apply(lambda x: [item for item in x if item not in noise])

without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))

songs_dataset['without_stop'] = without_stop

лемматизация

In [866]:
lemmatized = songs_dataset['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])

In [867]:
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))

In [868]:
songs_dataset['lemmatized'] = lemma

In [869]:
songs_dataset

,genre,lyrics,lowered,tokened,without_stop,lemmatized
0,Cristian,Who taught the sun where to stand in the morn...,who taught the sun where to stand in the morn...,"[who, taught, the, sun, where, to, stand, in, ...","taught, sun, stand, morning, ?, told, ocean, c...","taught, sun, stand, morning, ?, told, ocean, c..."
1,Rock,Foo Fighters The Colour And The Shape Everlong...,foo fighters the colour and the shape everlong...,"[foo, fighters, the, colour, and, the, shape, ...","foo, fighters, colour, shape, everlong, hello,...","foo, fighters, colour, shape, everlong, hello,..."


разделение на обучающую и тестовую выборки

In [870]:
x_train, x_test, y_train, y_test = train_test_split(songs_dataset.lemmatized, songs_dataset.genre, train_size = 0.7)

In [877]:
songs_dataset.genre.value_counts()

Cristian    1
Rock        1
Name: genre, dtype: int64

векторизация

In [875]:
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

классификация

In [872]:
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)

vectorized_x_test = vectorizer.transform(x_test)

In [873]:
clf.predict(vectorized_x_test)

array(['Cristian'], dtype='<U8')

оценка классификации

In [874]:
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    Cristian       0.00      0.00      0.00       0.0
        Rock       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1